# "Магические" методы

В Python многие объекты имеют так называемые «магические» атрибуты и
методы. Они имеют специальный формат имени, а именно, оно начинается и
заканчивается двумя подчеркиваниями. За это их еще называют
дандер-атрибутами. Вот некоторые из них:

- `__doc__` - хранит документацию класса, и используется при вызове
функции `help`;
- `__name__` - содержит имя класса;
- `__modle__` - содержит строку с именем модуля, или `'__main__'` если
модуль был запущен, а не импортирован;
- `__bases__` - содержит кортеж базовых классов, в котором всегда будет
элемент `<class 'object'>`;
- `__dict__` – словарь со всеми атрибутами класса;
- `__class__` - содержит ссылку на объект класса текущего объекта.

## Строковое представление

Подробнее рассмотрим специальные методы для управления строковым
представлением объекта.

Когда вы пытаетесь распечатать в консоли экземпляр какого-либо класса,
то, скорее всего, получите неудовлетворительный результат. По умолчанию
выводится только строка, содержащая имя класса и его уникальный
идентификатор.

In [7]:
class Point:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y

point = Point(0, 0)
print(f'{point = }')

point = <__main__.Point object at 0x0000014B8478B640>


Для более удобно представления объекта в виде строки существуют
дандер-методы `__repr__` и `__str__`. Метод `__str__` используется при
распечатке объекта через функцию `print` и `str` и предназначен для
восприятия человеком, т.е. его результат должен быть в первую очередь
удобочитаемым.

In [10]:
class Point:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'({self.x}, {self.y})'

point = Point(0, 0)
print(point)
print(f'{str(point) = }')

(0, 0)
str(point) = '(0, 0)'


В случае с методом `__repr__` идея состоит в том, что его результат
должен быть, прежде всего, однозначным. Его результат в основном
используется разработчиками и для отладки. При реализации этого метода
стоит ориентироваться на то, чтобы его результат можно было скопировать
и вставить в консоль и исполнить как фрагмент Python кода, который
вернет объект нужного класса. Также для этого метода есть специальная
функция `repr()`. Этот метод используется в f-строках. 

In [11]:
class Point:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y
    
    def __str__(self):
        return f'({self.x}, {self.y})'
    
    def __repr__(self):
        return f'{self.__class__.__name__}({self.x}, {self.y})'

point = Point(0, 0)
print(point)
print(f'{str(point) = }')
print(f'{point = }')

(0, 0)
str(point) = '(0, 0)'
point = Point(0, 0)


Методы `__repr__` и `__str__` имеют еще некоторые различия, с которыми
можно ознакомиться в документации. В заключение стоит порекомендовать
реализовывать хотя бы метод `__repr__` в своих классах, даже если он не
будет восстанавливать полное состояние объекта. Даже если опустить
реализацию `__str__`, то при его отсутствии будет вызван `__repr__`.

## Создание экземпляра

Создание экземпляра происходит в момент выполнения инструкции
`obj = Class()`. В Python создание экземпляра происходит в два шага.
Первым происходит создание непосредственно объекта экземпляра заданного
класса, после чего этот экземпляр инициализируется. Рассмотрим подробнее
эти шаги.

За создание нового объекта отвечает метод `__new__`. Этот метод
статический, это говорит о том, что первым аргументом он принимает
объект класса, однако применять соответствующий декоратор не требуется.
Он первым вызывается при исполнении инструкции `obj = Class()`.
Результатом выполнения `__new__` должен быть объект, обычно типа
`Class`. Следом за методом создания объекта экземпляра, но до
возвращения нового экземпляра пользователю, вызывается метод `__init__`,
который отвечает за инициализацию атрибутов экземпляра. Он является
методом экземпляра и первым аргументом принимает объект экземпляра.
Метод `__init__` возвращает `None`, любой отличный результат выполнения
будет порождать исключение `TypeError`. Рассмотрим на примере.

In [3]:
class Mayuri:
    def __new__(cls):
        print('Сотрудник лабораториии 002')
        return super().__new__(cls)
    
    def __init__(self):
        print('tutturu')


mayuri = Mayuri()

Сотрудник лабораториии 002
tutturu


Здесь при создании экземпляра `mayuri = Mayuri()` происходит сначала
вызов сначала `__new__`, затем `__init__`, что можно увидеть при выводе
сообщений. По своей сути конструкция `mayuri = Mayuri()` будет
эквивалентна:

In [5]:
mayuri = Mayuri.__new__(Mayuri)
mayuri.__init__()

Сотрудник лабораториии 002
tutturu


В большинстве случаев вам не нужно переопределять метод `__new__`.
В основном он нужен, когда вы хотите контролировать непосредственно
создание экземпляра, например, при реализации неизменяемых типов
(`str`, `int` и другие), а также различных паттернов, например,
синглтон (одиночка). Ниже привиден пример такого шаблона.

Суть этого шаблона в том, что мы переопределяем метод `__new__` для
проверки того, был ли создан экземпляр. В случае, если ранее экземпляр
уже создавался, повторного создания не происходит, а возвращается тот же
объект. Это удобно, когда в процессе работы программы нужно работать с
одним экземпляром, а хранить его в дополнительной переменной не очень
удобно.

In [6]:
class Singleton:
    # атрибут для хранения экземпляра
    # определим его как "приватный" по стандарту именования
    __instance = None

    def __new__(cls, *args, **kwargs):
        # проверяем был ли создан экземпляр ранее
        if not cls.__instance:
            # если нет, создаем новый и запоминаем
            cls.__instance = object.__new__(cls, *args, **kwargs)
        # возвращаем сохраненный экземпляр
        return cls.__instance


s1 = Singleton()
s2 = Singleton()
print(f'{s1 is s2 = }')

s1 is s2 = True


Обычно переопределять метод `__new__` не принято, но иногда это
требуется, если вы пишете API или модифицируете создание классов или
экземпляров или абстрагируете что-либо с помощью классов.

Интересное применение метода `__new__` можно найти в фреймворке Django,
например, в
[реализации модели (Осторожно! Необъяснимая магия!)](https://github.com/django/django/blob/ee46722cb9c860abec4d370adff052d0c1622d34/django/db/models/base.py#L72)

## Поиск атрибутов

Многие люди, переходящие на Python с других языков, жалуются, что ему не
хватает истинной инкапсуляции для классов; то есть невозможно определить
частные атрибуты с помощью общедоступных методов (геттеров и сеттеров).
На самом деле это не так. Python выполняет большую часть инкапсуляции
с помощью «магии», а не явных модификаторов для методов или атрибутов.
Одним из таких инструментов является механизм свойств о котором мы
поговорим в следующих главах. Сейчас же рассмотрим более глубогий или
"магичествие" методы управление поиском и доступом к атрибутам.

Для поиска и получения доступа к атрибутам в Python предусмотрен ряд
магических метов:
- `__getattr__(self, name)`
- `__setattr__(self, name, value)`
- `__delattr__(self, name)`
- `__getattribute__(self, name)`

Рассмотрим их более подробно. В примере класс `HolyGrail` содержит
только один атрибут с установленным значением. Магические методы, он
унаследовал от базового класса `object`, поэтому их поведение
стандартно. При обращении к атрибуту `shrubbery` будет возвращено его
значение, а обращение к несуществующим атрибам будет вызывать
исключение `AttributeError`.

In [13]:
class HolyGrail:
    def __init__(self):
        self.shrubbery = 1

hg = HolyGrail()
print(f'{hg.shrubbery = }')
print(f'{hg.holy_hand_grenade = }')

hg.shrubbery = 1


AttributeError: 'HolyGrail' object has no attribute 'holy_hand_grenade'

Добавим к этому кассу метод `__getattr__`. Он отвечает за поиск
атрибутов, которые еще не были объявлены. Python будет неявно вызывать
метод `__getattr__` всякий раз, когда вы запросите атрибут, который еще
не был определен. В Python есть аналог этого метода в виде функции
`getattr(object, name, default)`, которая под копотом вызывает
`__getattr__`, но дополнительно умеет возвращать значение по умолчанию.

В примере мы переопределили этот метод и реализовали
автоматическое добавление запрашиваемых атрибутов со значением по
умолчанию. Обратите внимание, что при обращении к атрибуту `shrubbery`
и повторному обращению к `holy_hand_grenade` метод `__getattr__`
вызываться не будет.

In [20]:
class HolyGrail:
    def __init__(self):
        self.shrubbery = 1

    def __getattr__(self, name):
        default_value = 42
        print(f'Попытка доступа к атрибуту {name}')
        if name not in self.__dict__:
            self.__dict__[name] = default_value
        return default_value


hg = HolyGrail()
print(f'{hg.holy_hand_grenade = }')
print(f'{hg.__dict__ = }')
print(f'{hg.shrubbery = }')

Попытка доступа к атрибуту holy_hand_grenade
hg.holy_hand_grenade = 42
hg.__dict__ = {'shrubbery': 1, 'holy_hand_grenade': 42}
hg.shrubbery = 1


Если метод `__getattr__` вызывается при попытке чтения атрибута, то
метод `__setattr__(self, name, value)` нужен для записи, т.е.
вызывается в момент исполнения инструкции `obj.name = value`. Этот
метод также вызывается неявно и имеет аналог в виде функции
`setattr(object, name, value)`.

Пример демонстрирует простейшую реализацию такого метода. Обратите
внимание, что метод `__setattr__` вызывается и при создании атрбутов в
методе `__init__`. Будте аккуратны с использованием этих методов, так
как довольно легко получить бесконечную рекурсию.

In [19]:
class HolyGrail:
    def __init__(self):
        self.shrubbery = 1

    def __setattr__(self, name, value):
        self.__dict__[name] = value
        print(f'Атрибут {name} добавлен со значением {value}')


hg = HolyGrail()
print(f'{hg.__dict__ = }')

hg.holy_hand_grenade = 42
print(f'{hg.holy_hand_grenade = }')
print(f'{hg.__dict__ = }')

Атрибут shrubbery добавлен со значением 1
hg.__dict__ = {'shrubbery': 1}
Атрибут holy_hand_grenade добавлен со значением 42
hg.holy_hand_grenade = 42
hg.__dict__ = {'shrubbery': 1, 'holy_hand_grenade': 42}


Метод `__delattr__(self, name)` предназначен для модификации процесса
удаления атрибута и вызывается при использовании конструкции
`del obj.name`.

В Python предоставляются гибкие возможности управления доступом, об этом
говорит наличие еще одного метода для обращения к атрибутам
`__getattribute__(self, name)`. В общих чертах его суть аналогична
методу `__getattr__`. Однако есть ряд существенных отличий.

Если класс реализует метод `__getattribute__`, Python вызывает этот
метод для каждого атрибута независимо от того, существует он или нет.
В этом и заключается его отличие от `__getattr__`. Одним из применениев
этого метода является контроль доступа к атрибутам класса, т.е.
реализация своего сокрятия.

Реализуем эту возможность для класса `HolyGrail`. Будем считать все
атрибуты, начинающиеся с префикса `'holy_'` приватными, и запретим к
ним доступ из вне.

In [22]:
class HolyGrail:
    def __init__(self):
        self.shrubbery = 1
        self.holy_hand_grenade = 42

    def __getattribute__(self, name: str):
        prefix = 'holy_'
        if name.startswith(prefix):
            raise AttributeError(f'Объект {self} не имеет атрибута {name}')
        return super().__getattribute__(name)

hg = HolyGrail()
print(f'{hg.shrubbery = }')
print(f'{hg.holy_hand_grenade = }')

hg.shrubbery = 1


AttributeError: Объект <__main__.HolyGrail object at 0x0000014B84DC8E50> не имеет атрибута holy_hand_grenade

Обратите внимание, что после проверки мы переиспользуем реализацию
метода `__getattribute__` родителя класса. Этот момент очень важен для
безопасного использования этого метода, иначе легко получить
бесконечную рекурсию.

Если класс реализует оба метода `__getattribute__` и `__getattr__`, то
первым будет вызван `__getattribute__`. Если он вернет исключение
`AttributeError`, то оно будет подавлено и будет вызван метод
`__getattr__`.

## Операторы сравнения

В Python все операции реализованы с помощью магических методов. Операции
сравнения не исключение. Каждый класс может реализовать следующий набор
методов для поддержки всех возможных операций сравнения:
- `__eq__(self, other)` - оператор `==`
- `__ne__(self, other)` - оператор `!=`
- `__lt__(self, other)` - оператор `<`
- `__gt__(self, other)` - оператор `>`
- `__le__(self, other)` - оператор `<=`
- `__ge__(self, other)` - оператор `>=`

Для упрощения реализации всех этих методов существует декоратор класса
`functools.total_ordering`. На основании метода `__eq__` и одного из
методов `__lt__`, `__le__`, `__gt__`, или `__ge__`, реализованных в
классе, он автоматически реализует все остальные методы.

## Арифметические операторы и не только

- `__invert__(self)` - оператор `~` инвертирования знака
- `__neg__(self)` - унарный минус
- `__pos__(self)` - унарный плюс
- `__round__(self, n)` - округление с помощью функции `round()`
- `__add__(self, other)` - сложение
- `__radd__(self, other)` - правое сложение
- `__sub__(self, other)` - вычитание
- `__mul__(self, other)` - умножение
- `__rmul__(self, other)` - правое умножение
- `__div__(self, other)` - деление
- `__mod__(self, other)` - оператор `%` остатка от деления
- `__or__(self, other)` - логический оператор `|`
- `__abs__` - модуль числа, функция `abs()`

Поддержка классом таких методов является реализацией протокола числа,
речь о которых пойдет в следующей главе.
Подробнее о таких методах читайте в
[документации](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types)

## Копирование

- `__copy__` - используется функцией `copy()` из модуля `copy`
- `__deepcopy` - используется функцией `deepcopy()` из модуля `copy`

## Другие полезные "магические" методы

- `__call__` - оператор круглые скобки
- `__contains__` - оператор `in`
- `__bool__` - преобразоание объекта в тип `bool`
- `__hash__` - вычисление хеша

# Полезные ссылки

- [Официальная документация о методе `__init__`](https://docs.python.org/3/reference/datamodel.html#object.__new__)
- [Unifying types and classes in Python 2.2](https://www.python.org/download/releases/2.2/descrintro/)
- [Why is `__init__()` always called after `__new__()`?](https://stackoverflow.com/questions/674304/why-is-init-always-called-after-new)
- [Зачем нужен `__new__` и каково его практическое применение?](https://ru.stackoverflow.com/questions/1025187/%D0%97%D0%B0%D1%87%D0%B5%D0%BC-%D0%BD%D1%83%D0%B6%D0%B5%D0%BD-new-%D0%B8-%D0%BA%D0%B0%D0%BA%D0%BE%D0%B2%D0%BE-%D0%B5%D0%B3%D0%BE-%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5-%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5)
- [When to use `__new__` vs. `__init__`?](https://mail.python.org/pipermail/tutor/2008-April/061426.html)
- [В двух словах о том, как работают метаклассы](https://twitter.com/1st1/status/1160956824822194178)
- [Python: `__new__` magic method explained](https://howto.lintel.in/python-__new__-magic-method-explained/)
- [Инстанцирование в Python](https://habr.com/ru/post/480022/)
- [`functools.total_ordering`](https://docs.python.org/3/library/functools.html)
- [A Guide to Python's Magic Methods](https://rszalski.github.io/magicmethods/)
- [Difference between `__getattr__` vs `__getattribute__`](https://stackoverflow.com/questions/3278077/difference-between-getattr-vs-getattribute)
- [Пользовательские атрибуты в Python](https://habr.com/ru/post/137415/)
